## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Importing Libraries</p>

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from mlxtend.classifier import StackingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from scipy.stats import mode
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import (
    roc_auc_score, roc_curve, auc, classification_report, 
    confusion_matrix, accuracy_score, matthews_corrcoef, f1_score,
    ConfusionMatrixDisplay
)
from sklearn.model_selection import (
    train_test_split, GridSearchCV, StratifiedShuffleSplit,
    StratifiedKFold, train_test_split, cross_val_score, cross_validate
)
import warnings
warnings.filterwarnings('ignore')

  ### <p style="background-color: #fdefff;color:#c12eff;display: inline-block;padding:.6rem;border-radius:.5rem;border: 1px solid #c059ff">Loading data</p>

In [2]:
cd /workspace/data/

/workspace/data


In [3]:
train_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/train.csv"))
test_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/test.csv"))
sample_submission_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/sample_submission.csv"))

print("train_data :", train_data.shape)
print("test_data :", test_data.shape)
print("sample_submission_data :", sample_submission_data.shape)

train_data : (3116945, 22)
test_data : (2077964, 21)
sample_submission_data : (2077964, 2)


## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Basic Info about Data</p> 

In [ ]:
train_data.head(2)

In [ ]:
test_data.head(2)

In [ ]:
sample_submission_data.head(2)

In [ ]:
train_data.info()

In [ ]:
train_data.describe().T

In [ ]:
print(train_data['class'].value_counts())
print(train_data['class'].value_counts() / sum(train_data['class'].value_counts()))
sns.countplot(x='class',data=train_data)
plt.xticks(rotation=60)
plt.show()

In [ ]:
pd.DataFrame({
    'column': train_data.columns,
    'null-count': train_data.isna().sum().values,
    '% null-count': np.round(train_data.isna().sum().values*100/len(train_data),6)
}).sort_values(by='null-count', ascending=False).reset_index(drop=True)

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Correlation Matrics</p> 

In [ ]:
# train_data.replace(['NaN', 'None', 'null', ''], np.nan, inplace=True)
df_dropped = train_data.drop('id', axis=1)
df_encoded = df_dropped.apply(lambda x: pd.factorize(x)[0] if x.dtype == 'object' else x)
train_data.isna().sum()

correlation_matrix = df_encoded.corr()

correlation_matrix
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Checking Unique Categories
</p> 

In [ ]:
# cate_col = train_data.select_dtypes(include=['object']).columns

# # Find unique categories and their counts for each categorical column
# unique_categories = {col: train_data[col].value_counts() for col in cate_col}

# # Set the size of the overall figure
# plt.figure(figsize=(15, len(cate_col) * 5))

# # Plot the count of each unique category
# for i, (col, counts) in enumerate(unique_categories.items(), 1):
#     plt.subplot(len(cate_col), 1, i)
#     sns.barplot(x=counts.index, y=counts.values, palette="viridis")
#     plt.title(f"Count of unique categories in column '{col}'")
#     plt.xlabel('Categories')
#     plt.ylabel('Count')
#     plt.xticks(rotation=45, ha='right')
#     plt.tight_layout()

# plt.show()


  ### <p style="background-color: #fdefff;color:#c12eff;display: inline-block;padding:.6rem;border-radius:.5rem;border: 1px solid #c059ff">Percentage of Missing Values by Feature</p>

In [ ]:
pd.DataFrame({
    'column': train_data.drop('id', axis=1).columns,
    'null-count': train_data.drop('id', axis=1).isna().sum().values,
    '% null-count': np.round(train_data.drop('id', axis=1).isna().sum().values*100/len(train_data.drop('id', axis=1)),6),
    'test null-count': test_data.isna().sum().values,
    '% test null-count': np.round(test_data.isna().sum().values*100/len(test_data),6)
}).sort_values(by='null-count', ascending=False).reset_index(drop=True)

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Checking Feature Importance and Dropping Useless Columns
</p> 

In [ ]:
# alpha = 0.05
# values = {}

# for col in train_data.columns:
#     if col == "class":
#         continue

#     A, B = train_data[col], train_data["class"]

#     dfObserved = pd.crosstab(A, B) 
#     chi2, p, dof, expected = scipy.stats.chi2_contingency(dfObserved.values) # 카이제곱 독립 검정 수행
#     values[col] = p
#     if p < alpha:
#         # Reject null hypothesis
#         print("{} is important. (p = {})".format(col, p))
#     else:
#         # Accept null hypothesis
#         print("{} is NOT important. (p = {})".format(col, p))

In [ ]:
# Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
#        'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
#        'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
#        'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
#        'habitat', 'season', 'veil-info'],
#       dtype='object')

# 결측치와 특성 존재 여부를 동시에 다룰 수 있음: 앞에서 0과 1로 veil-type의 존재 여부를 표현하고, 뒤에서 'unknown'을 통해 veil-color의 결측치를 처리함으로써 두 정보를 효과적으로 결합할 수 있습니다.
# 모델 학습에 유용한 정보 제공: 결측치나 특성의 존재 여부를 단순히 무시하지 않고, 이들을 결합하여 모델이 더 많은 패턴을 학습할 수 있게 도와줍니다.
# Index(['w', 'y', 'n', 'u', 'k', 'e', 'g', 'p', 'r', 'o', 's', 'a', 't', 'd',
#        'i', 'h', 'c', 'f', 'l', 'b', 'z', '8.25', '2.49', '3.32'],
#       dtype='object', name='veil-color')
# Index(['u', 'w', 'a', 'f', 'e', 'b', 'c', 'y', 'k', 'g', 'n', 's', 'r', 'd',
#        'p', 'h', 'i', 'l', 'is None', 't', '21.11', '5.94'],
#       dtype='object', name='veil-type')

# Index(['f10', 'tnone', 't1', 't7', 't10', 't4', 't2', 't5', 't3', 't9',
#        'fnone', 'f1', 'f2', 'f7', 'f4', 't8', 'f5', 'r2', 't6', 'f3', 'l4',
#        't0', 'p5', 'z7', 'c10', 'x10', 'f9', 'f6', 's10', 'm9', 'hnone', 's1',
#        'g3', 'g5', 'h7', 'e1', 'f0', 'r4', 'dnone', 's5', 'cnone', 'h1', 'p1',
#        'h5', 'h10', 'w3', 'y2', 'a10', 'ynone', 'e10', 'p7', '10.310', 's2',
#        'o2', 'g10', 'h2', 'g1', 's3', 'p3', 'knone', 'inone', 'nnone', 'rnone',
#        'l5', 'c1', 'n10', 'c3', 'o10', 'e4', 'd10', 'f has-ring10', 'lnone',
#        'c7', 'e3', 'y1', 'k10'],
#       dtype='object', name='has-ring-type')

train_data['veil-info'] = train_data['veil-type'].notna().astype(int).astype(str) + train_data['veil-color'].fillna('unknown')
cap_shape_mapping = {'b': 0, 'c': 1, 'x': 2, 'f': 3, 's': 4, 'p': 5, 'o': 6}
cap_color_mapping = {'n': 0, 'b': 1, 'g': 2, 'r': 3, 'p': 4, 'u': 5, 'e': 6, 'w': 7, 'y': 8, 'l': 9, 'o': 10, 'k': 11}
ring_type_mapping = {'c': 0, 'e': 1, 'r': 2, 'g': 3, 'l': 4, 'p': 5, 's': 6, 'z': 7, 'y': 8, 'm': 9, 'f': 10}
train_data['cap-shape'] = train_data['cap-shape'].map(cap_shape_mapping).astype(str).str.replace('.0', '', regex=False)
train_data['cap-color'] = train_data['cap-color'].map(cap_color_mapping).astype(str).str.replace('.0', '', regex=False)
train_data['has-ring-type'] = (
    train_data['has-ring'] + 
    train_data['ring-type'].fillna('none').map(ring_type_mapping).apply(lambda x: f"{x:.0f}" if pd.notna(x) else 'none').str.replace('.0', '', regex=False)
)

test_data['veil-info'] = test_data['veil-type'].notna().astype(int).astype(str) + test_data['veil-color'].fillna('unknown')
test_data['cap-shape'] = test_data['cap-shape'].map(cap_shape_mapping).astype(str).str.replace('.0', '', regex=False)
test_data['cap-color'] = test_data['cap-color'].map(cap_color_mapping).astype(str).str.replace('.0', '', regex=False)
test_data['has-ring-type'] = (
    test_data['has-ring'] + 
    test_data['ring-type'].fillna('none').map(ring_type_mapping).apply(lambda x: f"{x:.0f}" if pd.notna(x) else 'none').str.replace('.0', '', regex=False)
)

In [ ]:
train_data['has-ring-type']

In [ ]:
set(train_data['veil-info'])

In [ ]:
train_data = train_data.drop(['id', 'veil-color', 'veil-type', 'has-ring', 'ring-type'], axis=1)
test_data = test_data.drop(['id', 'veil-color', 'veil-type', 'has-ring', 'ring-type'], axis=1)

In [ ]:
# df_encoded = train_data
# dt_encoded = test_data

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Handling NaN Values And Less Frequent Categories</p> 

In [ ]:
# columns_to_fill_train = train_data.columns[(train_data.isna().sum() > 0) & (train_data.isna().sum() < 60)].tolist()
# columns_to_fill_test = test_data.columns[(test_data.isna().sum() > 0) & (test_data.isna().sum() < 100)].tolist()

# corr_matrix_train = df_encoded.corr()
# corr_matrix_test = dt_encoded.corr()

# def fill_missing_values(data, columns_to_fill, corr_matrix):
#     for column in columns_to_fill:
#         if column in corr_matrix.columns:
#             most_corr_features = corr_matrix[column][corr_matrix[column].abs() > 0.1].index.tolist()
#             if column in most_corr_features:
#                 most_corr_features.remove(column)

#             if most_corr_features:
#                 group_modes = data.groupby(most_corr_features)[column].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

#                 def fill_na(row):
#                     if pd.isna(row[column]):
#                         try:
#                             return group_modes.loc[tuple(row[most_corr_features])]
#                         except KeyError:
#                             return np.nan
#                     else:
#                         return row[column]

#                 data[column] = data.apply(fill_na, axis=1)

# fill_missing_values(df_encoded, columns_to_fill_train, corr_matrix_train)
# fill_missing_values(dt_encoded, columns_to_fill_test, corr_matrix_test)


In [ ]:
train_data.isna().sum()

In [ ]:
train_data.columns[(train_data.isna().sum() > 0) & (train_data.isna().sum() < 60)].tolist()

In [ ]:
def cleaning(df):
    threshold = 100
    
    cat_feats = ['cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'spore-print-color',
       'habitat', 'season', 'veil-info', 'has-ring-type']
    
    for feat in cat_feats:
        if df[feat].dtype.name == 'category':
            # Add 'missing' and 'noise' to categories if not present
            if 'missing' not in df[feat].cat.categories:
                df[feat] = df[feat].cat.add_categories('missing')
            if 'noise' not in df[feat].cat.categories:
                df[feat] = df[feat].cat.add_categories('noise')
        else:
            # Convert to category and add new categories
            df[feat] = df[feat].astype('category')
            df[feat] = df[feat].cat.add_categories(['missing', 'noise'])
        
        # Fill missing values with 'missing'
        df[feat] = df[feat].fillna('missing')
        
        # Replace infrequent categories with 'noise'
        counts = df[feat].value_counts(dropna=False)
        infrequent_categories = counts[counts < threshold].index
        df[feat] = df[feat].apply(lambda x: 'missing' if x in infrequent_categories else x)
    
    return df

# Example usage
train_data = cleaning(train_data)
test_data = cleaning(test_data)

In [ ]:
train_data.isna().sum()

In [ ]:
group_by_features = ['stem-width', 'stem-height']
group_means_train = train_data.groupby(group_by_features)['cap-diameter'].mean()
group_means_train

In [ ]:
group_by_features = ['stem-width', 'stem-height']
group_means_train = train_data.groupby(group_by_features)['cap-diameter'].mean()

def fill_na_with_group_mean(row):
    if pd.isna(row['cap-diameter']):
        group = tuple(row[group_by_features])
        return group_means_train.get(group, np.nan) 
    else:
        return row['cap-diameter']

train_data['cap-diameter'] = train_data.apply(fill_na_with_group_mean, axis=1)
test_data['cap-diameter'] = test_data.apply(fill_na_with_group_mean, axis=1)

In [ ]:
group_by_features = ['stem-width', 'stem-height']

# Calculate group means for the train data
group_means_train = train_data.groupby(group_by_features)['cap-diameter'].mean()

def fill_na_with_group_mean(row, group_means):
    if pd.isna(row['cap-diameter']):
        group = tuple(row[group_by_features])
        return group_means.get(group, np.nan)
    else:
        return row['cap-diameter']

# Apply to train_data using train group means
train_data['cap-diameter'] = train_data.apply(fill_na_with_group_mean, axis=1, group_means=group_means_train)

# Apply the same group means from train_data to test_data
test_data['cap-diameter'] = test_data.apply(fill_na_with_group_mean, axis=1, group_means=group_means_train)
###
###
# Calculate the mode from the training data
cap_diameter_mode = train_data['cap-diameter'].mode()[0]
stem_height_mode = train_data['stem-height'].mode()[0]

# Fill missing values in the training data using the mode calculated from the training data
train_data['cap-diameter'] = train_data['cap-diameter'].fillna(cap_diameter_mode)
train_data['stem-height'] = train_data['stem-height'].fillna(stem_height_mode)

# Fill missing values in the test data using the mode calculated from the training data
test_data['cap-diameter'] = test_data['cap-diameter'].fillna(cap_diameter_mode)
test_data['stem-height'] = test_data['stem-height'].fillna(stem_height_mode)

In [ ]:
# print("Nan Values in Train data")
# print(null_percent(train_data))
# print("Nan Values in Test data")
# print(null_percent(test_data))

In [ ]:
cat_feats = ['cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'spore-print-color',
       'habitat', 'season', 'veil-info', 'has-ring-type']

for feat in cat_feats:
    train_data[feat] = train_data[feat].astype('category')
for feat in cat_feats:
    test_data[feat] = test_data[feat].astype('category')

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Splitting Data</p> 

In [4]:
features = train_data.drop('class', axis=1).columns.to_list()
cat_feats = train_data[features].select_dtypes(include='object').columns.to_list()

def cleaner(df):
    for col in cat_feats:
        df[col] = df[col].fillna('missing')
        df.loc[df[col].value_counts(dropna=False)[df[col]].values < 100, col] = "noise"
        df[col] = df[col].astype('category')

    return df
    
train_data = cleaner(train_data)
test_data = cleaner(test_data)

cap_diameter_mean = train_data['cap-diameter'].mean(numeric_only=True)
train_data['cap-diameter'].fillna(cap_diameter_mean, inplace=True)
test_data['cap-diameter'].fillna(cap_diameter_mean, inplace=True)

In [5]:
X = train_data.drop(['class'], axis=1)
y = train_data['class']
print(X.shape, y.shape)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print(y[:3])

(3116945, 21) (3116945,)
[0 1 0]


## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Using XGBoost with optuna</p> 

In [ ]:
def model_report(estimator, X, y, cv=5):
    print("="*80)
    print(f"    Model: {estimator.__class__.__name__}")
    print("="*80)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=1/cv, shuffle=True, stratify=y, random_state=42)
    
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    print(f"F1 Score : {f1.mean():.6f}")
    print(f"MCC Score: {mcc.mean():.6f}")
    
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred)).plot()
    plt.title("Confusion Matrix")
    plt.show()

    print()

In [ ]:
# xgb_clf = XGBClassifier(enable_categorical=True, device="cuda", tree_method="hist")

# model_report(xgb_clf, X, y)

# cat_clf = CatBoostClassifier(
#     cat_features=cat_feats,
#     verbose=False,
#     allow_writing_files=False,
#     task_type="GPU"
# )

# model_report(cat_clf, X, y)

# lgb_clf = LGBMClassifier(device='cpu', verbosity=-1)

# model_report(lgb_clf, X, y)

In [7]:
# Callback to print the MCC score for each trial
def print_mcc_callback(study, trial):
    mcc = trial.user_attrs["mcc"]
    print(f"Trial {trial.number}: MCC = {mcc}")

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4885)

# Define the objective function for Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 2000, 3000),
        "eta": trial.suggest_float("eta", 1e-3, 1e-2),
        "gamma": trial.suggest_float("gamma", 0, 5.0),
        "max_depth": trial.suggest_int("max_depth", 2, 32),
        "min_child_weight": trial.suggest_int("min_child_weight", 40, 100),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "max_leaves": trial.suggest_int("max_leaves", 16, 84)
    }
    params['device'] = 'cuda'
    params['tree_method'] = 'hist'
    params['enable_categorical'] = True

    model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', enable_categorical=True, early_stopping_rounds=30) # , 
    # Fit the model with early stopping
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set
        verbose=False                 # Suppress output
    )
    
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc



storage = "sqlite:///xgb.db"

# Optimize hyperparameters with Optuna
study = optuna.create_study(storage=storage,
                            direction='maximize',
                           )
study.optimize(objective, n_trials=100, callbacks=[print_mcc_callback])

# Get the best parameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

[I 2024-08-29 20:34:44,837] A new study created in RDB with name: no-name-7e7d1661-eab7-4415-9029-a3635be359b3
[W 2024-08-29 20:34:44,877] Trial 0 failed with parameters: {'n_estimators': 2594, 'eta': 0.0025406968406256765, 'gamma': 1.738599598773523, 'max_depth': 21, 'min_child_weight': 61, 'subsample': 0.8579609993964666, 'colsample_bytree': 0.727496200357238, 'grow_policy': 'depthwise', 'max_leaves': 68} because of the following error: TypeError("xgboost.sklearn.XGBClassifier() got multiple values for keyword argument 'enable_categorical'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_4091/1659923731.py", line 20, in objective
    model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', enable_categorical=True, early_stopping_rounds=30) # ,
TypeError: xgboost.sklearn.XGBClassifier() got multiple values for keyword a

TypeError: xgboost.sklearn.XGBClassifier() got multiple values for keyword argument 'enable_categorical'

In [ ]:
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        "od_wait": trial.suggest_int("od_wait", 10, 50),
        "verbose": False,
        "allow_writing_files": False,
        "task_type": 'GPU',
        "cat_features": cat_feats
    }

    model = CatBoostClassifier(**params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set # ,
        verbose=False                 # Suppress output
    )
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc


study_name = "cat"
storage = "sqlite:///cat.db"

study = optuna.create_study(storage=storage,
                            study_name=study_name,
                            direction="maximize",
                            sampler=TPESampler(n_startup_trials=20, multivariate=True),
                            load_if_exists=True)

study.optimize(objective, n_trials=50, callbacks=[print_mcc_callback])

print(study.best_params)

In [9]:
def objective(trial):
    params = {
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "device": 'cpu',
        "verbosity": -1
    }

    model = LGBMClassifier(**params) # , early_stopping_rounds=30

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set
        # verbose=False                 # Suppress output
    )
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc


study_name = "lgb"
storage = "sqlite:///lgb.db"

study = optuna.create_study(storage=storage,
                            study_name=study_name,
                            direction="maximize",
                            sampler=TPESampler(n_startup_trials=20, multivariate=True),
                            load_if_exists=True)

study.optimize(objective, n_trials=100, callbacks=[print_mcc_callback]) # n_trials=100

print(study.best_params)

[I 2024-08-29 20:34:45,700] Using an existing study with name 'lgb' instead of creating a new one.
[I 2024-08-29 20:34:52,678] Trial 340 finished with value: 0.9845473359853243 and parameters: {'lambda_l1': 8.581056151436107e-06, 'lambda_l2': 0.0042026986062033145, 'num_leaves': 256, 'feature_fraction': 0.5166864805789477, 'bagging_fraction': 0.9139811568384277, 'bagging_freq': 2, 'min_child_samples': 77}. Best is trial 76 with value: 0.9848032057376603.


Trial 340: MCC = 0.9845473359853243


[I 2024-08-29 20:34:58,427] Trial 341 finished with value: 0.9845313275394644 and parameters: {'lambda_l1': 2.699104258553944e-08, 'lambda_l2': 0.0003826130404363176, 'num_leaves': 254, 'feature_fraction': 0.51691133545474, 'bagging_fraction': 0.8824545218100632, 'bagging_freq': 7, 'min_child_samples': 64}. Best is trial 76 with value: 0.9848032057376603.


Trial 341: MCC = 0.9845313275394644


[I 2024-08-29 20:35:03,352] Trial 342 finished with value: 0.9844796817833903 and parameters: {'lambda_l1': 1.2945072782857798e-05, 'lambda_l2': 0.005522377790588016, 'num_leaves': 234, 'feature_fraction': 0.43511708585154973, 'bagging_fraction': 0.8935195536812587, 'bagging_freq': 1, 'min_child_samples': 75}. Best is trial 76 with value: 0.9848032057376603.


...

Trial 437: MCC = 0.9846156411396481


[I 2024-08-29 20:43:25,424] Trial 438 finished with value: 0.9846055225831084 and parameters: {'lambda_l1': 7.160535705238856e-08, 'lambda_l2': 0.6748997233401334, 'num_leaves': 247, 'feature_fraction': 0.5936641734363114, 'bagging_fraction': 0.9276118284863495, 'bagging_freq': 1, 'min_child_samples': 91}. Best is trial 76 with value: 0.9848032057376603.


Trial 438: MCC = 0.9846055225831084


[I 2024-08-29 20:43:31,132] Trial 439 finished with value: 0.9845445099435982 and parameters: {'lambda_l1': 2.073874781938714e-07, 'lambda_l2': 0.020001397104344187, 'num_leaves': 205, 'feature_fraction': 0.4666263153404805, 'bagging_fraction': 0.9874266789970544, 'bagging_freq': 7, 'min_child_samples': 100}. Best is trial 76 with value: 0.9848032057376603.


Trial 439: MCC = 0.9845445099435982
{'lambda_l1': 1.3323938321586053e-07, 'lambda_l2': 0.07774032281247541, 'num_leaves': 248, 'feature_fraction': 0.41087472855265783, 'bagging_fraction': 0.9928696340850532, 'bagging_freq': 4, 'min_child_samples': 97}


In [ ]:
#  0.9849877748419223 
xgb_parameters={'n_estimators': 749, 'max_depth': 19, 'learning_rate': 0.08515243216339415, 'subsample': 0.8862800339551276, 'colsample_bytree': 0.6382531671740559, 'gamma': 2.1225041131747255e-07, 'lambda': 0.00018340832131612592, 'alpha': 7.642079671849836e-05, 'scale_pos_weight': 1.0077127113949682, 'eta': 0.008801461652478504, 'min_child_weight': 50, 'grow_policy': 'depthwise', 'max_leaves': 77,
               'device': 'cuda'
               }
# # 0.9848549793202802
# cat_parameters={'iterations': 1430, 'learning_rate': 0.08395155143710026, 'depth': 10, 'l2_leaf_reg': 1.7957745197789983, 'bootstrap_type': 'Bayesian', 'random_strength': 0.0010608689725419575, 'bagging_temperature': 0.40781319449377557, 'od_type': 'IncToDec', 'od_wait': 45,
#                 "task_type": 'GPU',
#                 "cat_features": cat_feats
#                }
# 0.9849163311592771
cat_parameters={'iterations': 2817, 'learning_rate': 0.07430964327633417, 'depth': 9, 'l2_leaf_reg': 4.105660971421866, 'bootstrap_type': 'Bayesian', 'random_strength': 4.279092407465261e-08, 'bagging_temperature': 0.3254799202354527, 'od_type': 'IncToDec', 'od_wait': 29,
                "task_type": 'GPU',
                "cat_features": cat_feats
               }
# 0.9848032057376603
lgb_parameters={'lambda_l1': 1.3323938321586053e-07, 'lambda_l2': 0.07774032281247541, 'num_leaves': 248, 'feature_fraction': 0.41087472855265783, 'bagging_fraction': 0.9928696340850532, 'bagging_freq': 4, 'min_child_samples': 97
               } 

In [ ]:
xgb_parameters = {
    'n_estimators': 2407,
    'eta': 0.009462133032592785,
    'gamma': 0.2865859948765318,
    'max_depth': 31,
    'min_child_weight': 47,
    'subsample': 0.6956431754146083,
    'colsample_bytree': 0.3670732604094118,
    'grow_policy': 'lossguide',
    'max_leaves': 73,
    'enable_categorical': True,
    'n_jobs': -1,
    'device': 'cuda',
    'tree_method': 'hist'
} # 0.9844272567086021

cat_parameters = {
    'iterations': 1041,
    'learning_rate': 0.08777255350163136,
    'depth': 10,
    'l2_leaf_reg': 0.1259643500248322,
    'bootstrap_type': 'Bayesian',
    'random_strength': 4.276181166674371e-08,
    'bagging_temperature': 0.35995482350907326,
    'od_type': 'Iter',
    'od_wait': 39,
    "verbose": False,
    "allow_writing_files": False,
    "task_type": 'GPU',
    "cat_features": cat_feats
} # 0.9841773055825763

lgb_parameters = {
    'n_estimators': 2500,
    'random_state':42,
    'max_bin':1024,
    'colsample_bytree':0.6,
    'reg_lambda': 80,
#     'device': 'gpu',
    'verbosity': -1
}

In [ ]:
def model_trainer(model, X, y, n_splits=5, random_state=101):
    skfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    oof_probs, oof_mccs = [], []
    print("="*80)
    print(f"Training {model.__class__.__name__}")
    print("="*80, end="\n")
    for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
        X_train, y_train = X.iloc[train_idx, :], y[train_idx]
        X_test, y_test = X.iloc[test_idx, :], y[test_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mcc = matthews_corrcoef(y_pred, y_test)
        oof_mccs.append(mcc)
        oof_probs.append(model.predict_proba(test_data))
        print(f"--- Fold {fold+1} MCC Score: {mcc:.6f}")
    print(f"\n---> Mean MCC Score: {np.mean(oof_mccs):.6f} \xb1 {np.std(oof_mccs):.6f}\n\n")
    return oof_probs, oof_mccs

In [ ]:
oof_probs = {}
oof_probs['xgb'], _ = model_trainer(XGBClassifier(**xgb_parameters), X, y, random_state=101)
oof_probs['cat'], _ = model_trainer(CatBoostClassifier(**cat_parameters), X, y, random_state=101)
oof_probs['lgb'], _ = model_trainer(LGBMClassifier(**lgb_parameters), X, y, random_state=101)

In [ ]:
oof_preds = {}
for model in oof_probs.keys():
    oof_preds[model] = np.argmax(np.mean(oof_probs[model], axis=0), axis=1)

In [ ]:
sub = pd.read_csv("./kaggle/playground-series-s4e8/sample_submission.csv")
preds = [pred for model, pred in oof_preds.items()]
md = mode(preds, axis=0)[0] if len(preds)>1 else preds[0]
sub['class'] = label_encoder.inverse_transform(md)
sub.to_csv('./kaggle/playground-series-s4e8/submission_xgb_cat_lgbm_0830_2.csv', index=False)